# Twitter Example

This notebook contains a heavily-commented minimal example of how to access Twitter using API keys and pull all tweets relating to a given search term.

The library used to interact with Twitter is [Tweepy](http://docs.tweepy.org/en/v3.5.0/index.html). This notebook explains only how to get Tweets for a single search term over the past seven days; if you want to do something more complicated - such as constantly update a file based on new tweets coming in in real time - the documentation linked above is the best place to look.

### 1. External setup

1. Install tweepy by running `conda install -c conda-forge tweepy` in the anaconda prompt

2. Create a [Twitter](https://twitter.com) account and verify it with a phone number

3. Apply for a developer account at [developer.twitter.com](https://developer.twitter.com)

4. Create an "app" on Twitter. This just means that you come up with a name and a justification for having Twitter API keys. "I want to experiment with Twitter's API" has always worked for me as a reason.

5. Give your app both read and write permissions.

### 2. Import libraries

In [1]:
import pandas as pd # Creates & manipulates dataframes
import tweepy # Interacts with Twitter

### 3. Authenticate a Twitter handler

When you create a Twitter app, you will be given a set of API keys. You should have the following:

- API key
- API secret key
- Access token
- Access token secret

In the code cell below, replace the placeholder values with your keys. Your keys should be inside speech marks, so the program will interpret them as strings.

In [2]:
API_KEY = ""
API_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

The cell below creates and authenticates an object - `API` - that can be used to interact with Twitter.

In [3]:
# Create an authentication handler
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)

# Feed the handler all the necessary information
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Use the handler to authenticate an API object
API = tweepy.API(auth, wait_on_rate_limit=True)

In the cell above, `wait_on_rate_limit` tells the `API` object not to make too many requests in a short space of time; this stops Twitter from flagging your program as harmful. Whenever `API` hits the request limit, it will stop making requests until the request counter has reset.

### 4. Access tweets relating to a specific search

Using the free API, you can only access Tweets from the last seven days.

In [4]:
# Make a holder to store tweets
tweets = []

# Create a string to search for
query='#datascience'

# Get tweets from the last seven days
for status in tweepy.Cursor(API.search,q=query).items():
    
    # Add each tweet to the holder
    tweets.append(status)

The cell above shows how to access Tweets using a cursor object. Essentially, a cursor allows you to page through search results on Twitter, getting one tweet at a time until there are no more to show.

The loop takes each tweet in turn as `status` and adds it to the holder, `tweets`.

In [5]:
# See how many tweets there are
len(tweets)

1185

### 5. Store the Tweets in a dataframe

Now we need to store our tweets in a dataframe, extracting out key information into columns.

Currently, all tweets are stored as Tweepy Status objects, containing not just the text but a huge number of values.

[An example of the Tweepy status object](https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2)

For the purposes of this example, only four different values are going to be extracted per tweet, but the same structure could be easily adapted to get more or different information.

The four values to be extracted per tweet are as follows:

- `created_at` - the date & time the tweet was posted
- `text` - the text of the tweet
- `retweet_count` - the number of times the tweet was retweeted
- `favourite_count` - the number of "favourites" a tweet has

In [6]:
# Make an empty dataframe with column headings

df = pd.DataFrame(columns=["created","text","retweets","favourites"])

# Inspect the empty dataframe
df.head()

,created,text,retweets,favourites


In [7]:
# Fill the dataframe with tweets by looping through each tweet and 
# attaching its information as a new row

# As many times as there are tweets in the holder,
for i in range(len(tweets)):
    
    # temporarily assign the next tweet to status
    status = tweets[i]
    
    # Make a new row in df with the desired values from status
    df.loc[i] = [status.created_at, status.text,
                 status.retweet_count, status.favorite_count]

In [8]:
# Check the dataframe
df.head()

,created,text,retweets,favourites
0,2019-07-13 12:10:06,The Ultimate Guide to Branding in 2019 https:/...,0,1
1,2019-07-13 12:10:02,RT @Ronald_vanLoon: 10 Free Must-Read Books fo...,2,0
2,2019-07-13 12:09:59,Get to insights quicker with Google Data Studi...,0,0
3,2019-07-13 12:09:59,Get to insights quicker with Google Data Studi...,0,0
4,2019-07-13 12:09:41,"RT Ronald_vanLoon ""10 Free Must-Read Books for...",0,0


### 6. Save the data as a .csv

Due to rate limits and the speed at which Tweets can be downloaded from Twitter, it's probably best to keep the code which accesses Twitter separate from the code that analyses Twitter data; that way, if you need to run the analysis sections multiple times, you don't have to wait for the data each time.

Because of this, the final section of this notebook simply stores the dataframe as a `.csv` file, allowing it to be loaded into other notebooks. 

In [9]:
# Save the whole dataframe to the same folder as the notebook file
df.to_csv("tweets.csv")